In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 KB 14.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=125b44035df242f00cd94be444e38d691429c04d53b3afff2232cb0ae35c8399
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5


In [2]:
!pip install findspark

In [3]:
import findspark
findspark.init

<function findspark.init(spark_home=None, python_path=None, edit_rc=False, edit_profile=False)>

In [4]:
from pyspark.sql import SparkSession # required to created a dataframe
spark = SparkSession.builder.appName("Basics").getOrCreate()

import pyspark.sql.functions as F

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/09 08:14:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [36]:
df = spark.read.csv("../input/heart-failure-prediction/heart.csv",
                    header=True,
                   inferSchema=True)

# read csv, all columns will be of type string
# df = spark.read.option('header','true').csv('heart.csv')
# tell pyspark the type of the columns - saves time on large dataset. there are other ways to do this, but that's
# my favorite
# schema = 'Age INTEGER, Sex STRING, ChestPainType STRING'
# df = spark.read.csv('../input/heart-failure-prediction/heart.csv', schema=schema, inferSchema=True ,header=True)
# let PySpark infer the schema
# df = spark.read.csv('../input/heart-failure-prediction/heart.csv', inferSchema=True, header=True)
# # replace nulls with other value at reading time
# df = spark.read.csv('../input/heart-failure-prediction/heart.csv', nullValue='NA')
# save data
# df.write.format("csv").save("./heart_save.csv")
# # if you want to overwrite the file
# df.write.format("csv").mode("overwrite").save("./heart_save.csv")

df.show()

+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| 40|  M|          ATA|      140|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|
| 49|  F|          NAP|      160|        180|        0|    Normal|  156|             N|    1.0|    Flat|           1|
| 37|  M|          ATA|      130|        283|        0|        ST|   98|             N|    0.0|      Up|           0|
| 48|  F|          ASY|      138|        214|        0|    Normal|  108|             Y|    1.5|    Flat|           1|
| 54|  M|          NAP|      150|        195|        0|    Normal|  122|             N|    0.0|      Up|           0|
| 39|  M|          NAP|      120|        339|        0| 

In [37]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- ChestPainType: string (nullable = true)
 |-- RestingBP: integer (nullable = true)
 |-- Cholesterol: integer (nullable = true)
 |-- FastingBS: integer (nullable = true)
 |-- RestingECG: string (nullable = true)
 |-- MaxHR: integer (nullable = true)
 |-- ExerciseAngina: string (nullable = true)
 |-- Oldpeak: double (nullable = true)
 |-- ST_Slope: string (nullable = true)
 |-- HeartDisease: integer (nullable = true)



In [38]:
df.count()

918

# Pandas DataFrame VS PySpark DataFrame

**both represents a table of data with rows and columns. however, under the hood they are different, as PySpark dataframe needs to support distributed computations. as we move forward, we will see more and more features of it that are not present in Pandas DataFrame. that being said - if you know how to use Pandas, than moving to PySpark will feel like a natural transition.**

# DAG

**directed acyclic graph is the way Spark runs computations. when you give it a series of transformation to apply to the dataset, it build a graph out of those transformations, so it knows what to do - but it does not execute those commands immediately, if it does not have to. rather, it is lazy - it will go through the DAG and apply the transformations only when it must, to provide a needed result. this allows better performance, since spark knows what's ahead of a certain computation and get optimize the process accordingly.**

# transformations VS actions

**in PySpark, there are two types of command: transformations and actions. transformation commands are added to the DAG, but does not get it to actually be executed. they transform one DataFrame into another, not changing the input DataFrame. on the other hand, actions make PySpark execute the DAG but does not create a new DataFrame - instead, they output the result of the DAG.**

# Caching

**every time you run a DAG, it will be re-computed from the beginning. that is, the results are not saved in memory. so, if we want to save a result so it won't have to be recomputed, we can use the cache command. note, that this will occupy space in the working node's memory - so be careful with the sizes of datasets you are caching! by default, the cached DF is stored to RAM, and is unserialized (not converted into a stream of bytes). you can change both of these - store data to hard disk, serialized it, or both!**

# Collecting

**even after caching a DataFrame, it still sits in the worker nodes memory. if you want to collect its pieces, assemble them and save them on the master node so you won't have to pull it every time, use the command for collecting. again, be very careful with this, since the collected file will have to fit in the master node memory!**

In [39]:
df.cache()
df.collect()

22/05/09 09:06:47 WARN CacheManager: Asked to cache already cached data.


[Row(Age=40, Sex='M', ChestPainType='ATA', RestingBP=140, Cholesterol=289, FastingBS=0, RestingECG='Normal', MaxHR=172, ExerciseAngina='N', Oldpeak=0.0, ST_Slope='Up', HeartDisease=0),
 Row(Age=49, Sex='F', ChestPainType='NAP', RestingBP=160, Cholesterol=180, FastingBS=0, RestingECG='Normal', MaxHR=156, ExerciseAngina='N', Oldpeak=1.0, ST_Slope='Flat', HeartDisease=1),
 Row(Age=37, Sex='M', ChestPainType='ATA', RestingBP=130, Cholesterol=283, FastingBS=0, RestingECG='ST', MaxHR=98, ExerciseAngina='N', Oldpeak=0.0, ST_Slope='Up', HeartDisease=0),
 Row(Age=48, Sex='F', ChestPainType='ASY', RestingBP=138, Cholesterol=214, FastingBS=0, RestingECG='Normal', MaxHR=108, ExerciseAngina='Y', Oldpeak=1.5, ST_Slope='Flat', HeartDisease=1),
 Row(Age=54, Sex='M', ChestPainType='NAP', RestingBP=150, Cholesterol=195, FastingBS=0, RestingECG='Normal', MaxHR=122, ExerciseAngina='N', Oldpeak=0.0, ST_Slope='Up', HeartDisease=0),
 Row(Age=39, Sex='M', ChestPainType='NAP', RestingBP=120, Cholesterol=339, F

In [40]:
# convert PySpark DataFrame to Pandas DataFrame
pd_df = df.toPandas()
# convert it back
spark_df = spark.createDataFrame(pd_df)

In [41]:
# show first three rows as three row objects, which is how spark represents single rows from a table.
# we will learn more about it later
df.head(3)

[Row(Age=40, Sex='M', ChestPainType='ATA', RestingBP=140, Cholesterol=289, FastingBS=0, RestingECG='Normal', MaxHR=172, ExerciseAngina='N', Oldpeak=0.0, ST_Slope='Up', HeartDisease=0),
 Row(Age=49, Sex='F', ChestPainType='NAP', RestingBP=160, Cholesterol=180, FastingBS=0, RestingECG='Normal', MaxHR=156, ExerciseAngina='N', Oldpeak=1.0, ST_Slope='Flat', HeartDisease=1),
 Row(Age=37, Sex='M', ChestPainType='ATA', RestingBP=130, Cholesterol=283, FastingBS=0, RestingECG='ST', MaxHR=98, ExerciseAngina='N', Oldpeak=0.0, ST_Slope='Up', HeartDisease=0)]

In [42]:
df.dtypes

[('Age', 'int'),
 ('Sex', 'string'),
 ('ChestPainType', 'string'),
 ('RestingBP', 'int'),
 ('Cholesterol', 'int'),
 ('FastingBS', 'int'),
 ('RestingECG', 'string'),
 ('MaxHR', 'int'),
 ('ExerciseAngina', 'string'),
 ('Oldpeak', 'double'),
 ('ST_Slope', 'string'),
 ('HeartDisease', 'int')]

In [43]:
# cast a column from one type to other
from pyspark.sql.types import FloatType
df = df.withColumn("Age",df.Age.cast(FloatType()))
df = df.withColumn("RestingBP",df.Age.cast(FloatType()))

In [44]:
# compute summery statistics
df.select(['Age','RestingBP']).describe().show()

+-------+------------------+------------------+
|summary|               Age|         RestingBP|
+-------+------------------+------------------+
|  count|               918|               918|
|   mean|53.510893246187365|53.510893246187365|
| stddev|  9.43261650673202|  9.43261650673202|
|    min|              28.0|              28.0|
|    max|              77.0|              77.0|
+-------+------------------+------------------+



In [45]:
# add a new column or replace existing one
AgeFixed = df['Age'] + 1  # select alwayes returns a DataFrame object, and we need a column object
df = df.withColumn('AgeFixed', AgeFixed)

In [46]:
df.select(['AgeFixed','Age']).describe().show()

+-------+------------------+------------------+
|summary|          AgeFixed|               Age|
+-------+------------------+------------------+
|  count|               918|               918|
|   mean|54.510893246187365|53.510893246187365|
| stddev|  9.43261650673202|  9.43261650673202|
|    min|              29.0|              28.0|
|    max|              78.0|              77.0|
+-------+------------------+------------------+



In [47]:
# remove columns
df.drop('AgeFixed').show(1) # add df = to get the new DataFrame into a variable

+----+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
| Age|Sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
+----+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|40.0|  M|          ATA|     40.0|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|
+----+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
only showing top 1 row



In [50]:
# rename a column
# df.withColumnRenamed('Age','age').select('age').show(1)
# to rename more than a single column, i would suggest a loop.
name_pairs = [('Age','age'),('Sex','sex')]
for old_name, new_name in name_pairs:
    df = df.withColumnRenamed(old_name,new_name)

In [51]:
df.show(1)

+----+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+--------+
| age|sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|AgeFixed|
+----+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+--------+
|40.0|  M|          ATA|     40.0|        289|        0|    Normal|  172|             N|    0.0|      Up|           0|    41.0|
+----+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+--------+
only showing top 1 row



In [54]:
# drop all rows that contain any NA
df = df.na.drop()
df.count()
# drop all rows where all values are NA
df = df.na.drop(how='all')
# drop all rows where more at least 2 values are NOT NA
df = df.na.drop(thresh=2)
# drop all rows where any value at specific columns are NAs.
df = df.na.drop(how='any', subset=['age','sex']) # 'any' is the defult

In [55]:
# fill missing values in a specific column with a '?'
df = df.na.fill(value='?',subset=['sex'])
# replace NAs with mean of column
from pyspark.ml.feature import Imputer # In statistics, imputation is the process of
                                       # replacing missing data with substituted values
imptr = Imputer(inputCols=['age','RestingBP'],
                outputCols=['age','RestingBP']).setStrategy('mean') # can also be 'median' and so on

df = imptr.fit(df).transform(df)

In [63]:
# filter to adults only and calculate mean
df.filter('age > 18').select('age').describe().show()
df.where('age > 18')# 'where' is an alias to 'filter'
df.where(df['age'] > 18) # third option
# add another condition ('&' means and, '|' means or)
df.where((df['age'] > 18) | (df['ChestPainType'] == 'ATA'))
# take every record where the 'ChestPainType' is NOT 'ATA'
df.filter(~(df['ChestPainType'] == 'ATA'))

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|               918|
|   mean|53.510893246187365|
| stddev|  9.43261650673202|
|    min|              28.0|
|    max|              77.0|
+-------+------------------+



DataFrame[age: float, sex: string, ChestPainType: string, RestingBP: float, Cholesterol: int, FastingBS: int, RestingECG: string, MaxHR: int, ExerciseAngina: string, Oldpeak: double, ST_Slope: string, HeartDisease: int, AgeFixed: float]

In [64]:
# evaluate a string expression into command
from pyspark.sql.functions import expr
exp = 'age + 0.2 * AgeFixed'
df.withColumn('new_col', expr(exp)).select('new_col').show(3)

+-------+
|new_col|
+-------+
|   48.2|
|   59.0|
|   44.6|
+-------+
only showing top 3 rows



In [69]:
# group by age
disease_by_age = df.groupby('age').mean().select(['age','avg(HeartDisease)'])
# sort values in desnding order
from pyspark.sql.functions import desc
disease_by_age.orderBy(desc("age")).show(5)

+----+------------------+
| age| avg(HeartDisease)|
+----+------------------+
|77.0|               1.0|
|76.0|               0.5|
|75.0|0.6666666666666666|
|74.0|0.7142857142857143|
|73.0|               1.0|
+----+------------------+
only showing top 5 rows



In [70]:
# aggregate to get several statistics for several columns
# the available aggregate functions are avg, max, min, sum, count
from pyspark.sql import functions as F
df.agg(F.min(df['age']),F.max(df['age']),F.avg(df['sex'])).show()

+--------+--------+--------+
|min(age)|max(age)|avg(sex)|
+--------+--------+--------+
|    28.0|    77.0|    null|
+--------+--------+--------+



In [71]:
df.groupby('HeartDisease').agg(F.min(df['age']),F.avg(df['sex'])).show()

+------------+--------+--------+
|HeartDisease|min(age)|avg(sex)|
+------------+--------+--------+
|           1|    31.0|    null|
|           0|    28.0|    null|
+------------+--------+--------+



In [72]:
# run an SQL query on the data
df.createOrReplaceTempView("df") # tell PySpark how the table will be called in the SQL query
spark.sql("""SELECT sex from df""").show(2)

# we also choose columns using SQL sytnx, with a command that combins '.select()' and '.sql()'
df.selectExpr("age >= 40 as older", "age").show(2)

+---+
|sex|
+---+
|  M|
|  F|
+---+
only showing top 2 rows

+-----+----+
|older| age|
+-----+----+
| true|40.0|
| true|49.0|
+-----+----+
only showing top 2 rows



In [77]:
df.groupby('age').pivot('sex', ("M", "F")).count().show(3)

+----+---+---+
| age|  M|  F|
+----+---+---+
|64.0| 16|  6|
|47.0| 15|  4|
|58.0| 35|  7|
+----+---+---+
only showing top 3 rows



In [88]:
# pivot - expensive operation
df.selectExpr("age >= 40 as older", "age",'sex').groupBy("sex")\
                    .pivot("older", ("true", "false")).count().show()

+---+----+-----+
|sex|true|false|
+---+----+-----+
|  F| 174|   19|
|  M| 664|   61|
+---+----+-----+



In [89]:
df.select(['age','MaxHR','Cholesterol']).show(4)

+----+-----+-----------+
| age|MaxHR|Cholesterol|
+----+-----+-----------+
|40.0|  172|        289|
|49.0|  156|        180|
|37.0|   98|        283|
|48.0|  108|        214|
+----+-----+-----------+
only showing top 4 rows



In [94]:
df.printSchema()

root
 |-- age: float (nullable = true)
 |-- sex: string (nullable = false)
 |-- ChestPainType: string (nullable = true)
 |-- RestingBP: float (nullable = true)
 |-- Cholesterol: integer (nullable = true)
 |-- FastingBS: integer (nullable = true)
 |-- RestingECG: string (nullable = true)
 |-- MaxHR: integer (nullable = true)
 |-- ExerciseAngina: string (nullable = true)
 |-- Oldpeak: double (nullable = true)
 |-- ST_Slope: string (nullable = true)
 |-- HeartDisease: integer (nullable = true)
 |-- AgeFixed: float (nullable = true)



In [103]:
# devide dataset to training features and target
X_column_names = ['age','Cholesterol']
target_colum_name = ['MaxHR']

# convert feature columns into a columns where the vlues are feature vectors
from pyspark.ml.feature import VectorAssembler
v_asmblr = VectorAssembler(inputCols = X_column_names, outputCol = 'Fvec')
df = v_asmblr.transform(df)
X = df.select(['age','Cholesterol','Fvec','MaxHR'])
X.show(3)

+----+-----------+------------+-----+
| age|Cholesterol|        Fvec|MaxHR|
+----+-----------+------------+-----+
|40.0|        289|[40.0,289.0]|  172|
|49.0|        180|[49.0,180.0]|  156|
|37.0|        283|[37.0,283.0]|   98|
+----+-----------+------------+-----+
only showing top 3 rows



In [104]:
# devide dataset into training and testing sets
trainset, testset = X.randomSplit([0.8,0.2])

In [105]:
# predict 'RestingBP' using linear regression
from pyspark.ml.regression import LinearRegression
model = LinearRegression(featuresCol='Fvec', labelCol='MaxHR')
model = model.fit(trainset)
print(model.coefficients)
print(model.intercept)

22/05/09 10:31:27 WARN Instrumentation: [8ca21c76] regParam is zero, which might cause numerical instability and overfitting.


[-0.9502248437685721,0.049563607225588405]
177.2694046433669


In [106]:
# evaluate model
model.evaluate(testset).predictions.show(3)

/opt/conda/lib/python3.7/site-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+----+-----------+------------+-----+------------------+
| age|Cholesterol|        Fvec|MaxHR|        prediction|
+----+-----------+------------+-----+------------------+
|29.0|        204|[29.0,204.0]|  202|159.82386004809834|
|29.0|        263|[29.0,263.0]|  170|162.74811287440806|
|32.0|        198|[32.0,198.0]|  165| 156.6758038734391|
+----+-----------+------------+-----+------------------+
only showing top 3 rows



In [111]:
# handel categorical features with ordinal indexing
from pyspark.ml.feature import StringIndexer
indxr = StringIndexer(inputCol='ChestPainType', outputCol='ChestPainTypeInxed')
indxr.fit(df).transform(df).select('ChestPainTypeInxed').show(3)

+------------------+
|ChestPainTypeInxed|
+------------------+
|               2.0|
|               1.0|
|               2.0|
+------------------+
only showing top 3 rows

